In [1]:
import numpy as np
import pandas as pd
from  scipy.interpolate  import  interp1d
import itertools

# $f$ Value

## 1. $f_1$ and $f_2$ value

Based on the table of $l$ value and $f$ value, calculate $f$ value coresponding to 'Glass type', 'Heating and cooling seasons', 'Region' and 'Direction'.  
When there is no $l$ value in the table, calculate by interpolating. In case the $l$ value is over 20.0, consier the $l$ value equal to 20.0.

In [2]:
class f1f2Value:

    # obtain the list from csv data
    def __init__(self):
        self.f_list = pd.read_csv('fValue.csv', skiprows = 1)
        self.f_list['ガラス仕様の区分'] = self.f_list['ガラス仕様の区分'].fillna(method='ffill')
        self.f_list['期間']             = self.f_list['期間']            .fillna(method='ffill')  
        self.f_list.columns=['Glass Spec',
                             'Season',
                             'l_value',
                             '1N','1NE','1E','1SE','1S','1SW','1W','1NW',
                             '2N','2NE','2E','2SE','2S','2SW','2W','2NW',
                             '3N','3NE','3E','3SE','3S','3SW','3W','3NW',
                             '4N','4NE','4E','4SE','4S','4SW','4W','4NW',
                             '5N','5NE','5E','5SE','5S','5SW','5W','5NW',
                             '6N','6NE','6E','6SE','6S','6SW','6W','6NW',
                             '7N','7NE','7E','7SE','7S','7SW','7W','7NW',
                             '8N','8NE','8E','8SE','8S','8SW','8W','8NW']
        self.f_list['Season'] = self.f_list['Season'].replace( { '冷房':'Cooling', '暖房':'Heating' } )
        self.f_list['Glass Spec'] = self.f_list['Glass Spec'].replace( { 1:'Type1', 2:'Type2', 3:'Type3', 4:'Type4', 5:'Type5', 6:'Type6', 7:'Type7' } )

    # get l and nu list in table
    def getTableValue(self, glassType, season, region, direction):
        r = str({ 'region1':1, 'region2':2, 'region3':3, 'region4':4, 'region5':5, 'region6':6, 'region7':7, 'region8':8 }[region])
        d = { 'N':'N', 'NE':'NE', 'E':'E', 'SE':'SE', 'S':'S', 'SW':'SW', 'W':'W', 'NW':'NW' }[direction]
        return self.f_list[ (self.f_list['Season']==season) & (self.f_list['Glass Spec']==glassType) ].loc[:,['l_value',r+d]]

    # get f1 or f2 value
    def getValue(self, glassType, season, region, direction, lValue):
        if lValue > 20.0:
            lValue = 20.0
        dat = self.getTableValue(glassType, season, region, direction)
        return float( interp1d(dat.iloc[:,0],dat.iloc[:,1])(lValue) )

#### Example - Read Data

In [3]:
v = f1f2Value()
v.f_list

,Glass Spec,Season,l_value,1N,1NE,1E,1SE,1S,1SW,1W,...,7W,7NW,8N,8NE,8E,8SE,8S,8SW,8W,8NW
0,Type1,Cooling,0.0,0.139,0.106,0.084,0.081,0.091,0.087,0.090,...,0.092,0.112,0.141,0.11,0.089,0.086,0.095,0.088,0.09,0.111
1,Type1,Cooling,0.4,0.260,0.229,0.197,0.159,0.169,0.168,0.204,...,0.202,0.232,0.264,0.229,0.203,0.18,0.178,0.185,0.21,0.233
2,Type1,Cooling,0.5,0.293,0.266,0.231,0.181,0.188,0.190,0.237,...,0.231,0.262,0.294,0.258,0.236,0.207,0.199,0.211,0.243,0.263
3,Type1,Cooling,0.6,0.324,0.298,0.267,0.205,0.207,0.214,0.272,...,0.265,0.289,0.323,0.285,0.269,0.233,0.219,0.238,0.276,0.292
4,Type1,Cooling,0.7,0.353,0.326,0.301,0.229,0.224,0.238,0.305,...,0.296,0.321,0.35,0.313,0.3,0.261,0.24,0.265,0.308,0.32
5,Type1,Cooling,0.8,0.380,0.354,0.333,0.255,0.241,0.263,0.336,...,0.325,0.350,0.376,0.343,0.33,0.288,0.26,0.292,0.337,0.351
6,Type1,Cooling,1.0,0.428,0.411,0.394,0.308,0.272,0.313,0.395,...,0.384,0.398,0.422,0.393,0.388,0.339,0.304,0.341,0.394,0.401
7,Type1,Cooling,1.2,0.470,0.457,0.443,0.358,0.298,0.361,0.443,...,0.432,0.441,0.463,0.436,0.436,0.386,0.349,0.387,0.442,0.442
8,Type1,Cooling,1.4,0.505,0.494,0.487,0.405,0.320,0.407,0.486,...,0.475,0.483,0.499,0.476,0.478,0.429,0.39,0.428,0.482,0.481
9,Type1,Cooling,1.6,0.536,0.526,0.526,0.446,0.342,0.446,0.525,...,0.513,0.517,0.53,0.51,0.517,0.466,0.427,0.465,0.519,0.516


#### Example - Get L Value and nu Value Table

In [4]:
v.getTableValue(glassType = 'Type1', season = 'Cooling', region = 'region1', direction = 'N')

,l_value,1N
0,0.0,0.139
1,0.4,0.260
2,0.5,0.293
3,0.6,0.324
4,0.7,0.353
5,0.8,0.380
6,1.0,0.428
7,1.2,0.470
8,1.4,0.505
9,1.6,0.536


#### Example - Get L Value for Heating

Season : Heating  
l-value : 1/0.3

In [5]:
print('region origin value')
for r, d in itertools.product( ['region1','region2','region3','region4','region5','region6','region7'],
                                    ['SW','NW','NE','SE'] ):
        f_H = v.getValue( 'Type7', 'Heating', r, d, 1/0.3 )
        print( r, d, np.round(f_H,3) )

region origin value
region1 SW 0.658
region1 NW 0.605
region1 NE 0.602
region1 SE 0.665
region2 SW 0.648
region2 NW 0.604
region2 NE 0.605
region2 SE 0.656
region3 SW 0.65
region3 NW 0.602
region3 NE 0.602
region3 SE 0.649
region4 SW 0.642
region4 NW 0.595
region4 NE 0.598
region4 SE 0.642
region5 SW 0.667
region5 NW 0.595
region5 NE 0.592
region5 SE 0.671
region6 SW 0.654
region6 NW 0.595
region6 NE 0.589
region6 SE 0.675
region7 SW 0.67
region7 NW 0.589
region7 NE 0.587
region7 SE 0.666


#### Example - Get L Value for Cooling

Season : Cooling  
l-value : 20.0

In [6]:
print('region origin value')
for r, d in itertools.product( ['region1','region2','region3','region4','region5','region6','region7','region8'],
                                    ['SW','NW','NE','SE'] ):
    f_C = v.getValue( 'Type1', 'Cooling', r, d, 20.0 )
    print( r, d, np.round(f_C,3) )

region origin value
region1 SW 0.86
region1 NW 0.866
region1 NE 0.865
region1 SE 0.864
region2 SW 0.861
region2 NW 0.864
region2 NE 0.864
region2 SE 0.858
region3 SW 0.859
region3 NW 0.865
region3 NE 0.862
region3 SE 0.853
region4 SW 0.85
region4 NW 0.861
region4 NE 0.861
region4 SE 0.853
region5 SW 0.858
region5 NW 0.862
region5 NE 0.863
region5 SE 0.854
region6 SW 0.852
region6 NW 0.864
region6 NE 0.862
region6 SE 0.852
region7 SW 0.847
region7 NW 0.862
region7 NE 0.861
region7 SE 0.849
region8 SW 0.858
region8 NW 0.859
region8 NE 0.857
region8 SE 0.86
